<a href="https://colab.research.google.com/github/FabioBoccia/Progetto_ESM/blob/dev/Progetto_Bozza_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%rm -r sample_data
!wget --user=corso --password=p2021corso http://www.grip.unina.it/download/corso/ffhq_real.zip
!unzip -q ffhq_real.zip
!mkdir -p ffhq/train/real
!mkdir -p ffhq/train/synthesized
!mkdir -p ffhq/test/real
!mkdir -p ffhq/test/synthesized
%cd /content/0_real/
!mv $(ls | shuf -n 2500) ../ffhq/train/real/
!mv * ../ffhq/test/real/
%cd ..
!rm -d 0_real

rm: cannot remove 'sample_data': No such file or directory
--2022-05-18 19:56:31--  http://www.grip.unina.it/download/corso/ffhq_real.zip
Resolving www.grip.unina.it (www.grip.unina.it)... 143.225.28.237
Connecting to www.grip.unina.it (www.grip.unina.it)|143.225.28.237|:80... connected.
HTTP request sent, awaiting response... 401 Unauthorized
Authentication selected: Basic realm="corso"
Reusing existing connection to www.grip.unina.it:80.
HTTP request sent, awaiting response... 200 OK
Length: 4072216854 (3.8G) [application/zip]
Saving to: ‘ffhq_real.zip.1’

ffhq_real.zip.1       0%[                    ]       0  --.-KB/s               ^C
/content/0_real
mv: cannot stat '*': No such file or directory
/content


In [2]:
!cp ffhq/train/real/* ffhq/train/synthesized/
!cp ffhq/test/real/* ffhq/test/synthesized/

^C


In [3]:
!%reset -f
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io
import tensorflow as tf

/bin/bash: line 0: fg: no job control


In [4]:
batch_size = 32
interpolation = 'bilinear'
train_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=1/5, # Validation is 1/6 of the total, so 1/5 of (total - test)
    subset='training',
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)
val_dataset = tf.keras.utils.image_dataset_from_directory(
    'ffhq/train/',
    labels='inferred',
    label_mode='binary',
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(256, 256),
    shuffle=True,
    seed=125,
    validation_split=1/5,
    subset='validation',
    interpolation=interpolation,
    crop_to_aspect_ratio=True,
)

Found 5063 files belonging to 2 classes.
Using 4051 files for training.
Found 5063 files belonging to 2 classes.
Using 1012 files for validation.


In [5]:
k = tf.random.uniform(
        shape=[],
        minval=0.0,
        maxval=3.0,
        dtype=tf.dtypes.float32
    )
print(k)

tf.Tensor(2.487059, shape=(), dtype=float32)


In [6]:
# Preprocessing
!pip install tensorflow_io

import tensorflow_io as tfio
d = 3
def randomGaussianBlur(x):
    k = tf.random.uniform(
        shape=[],
        minval=0.0,
        maxval=3.0,
        dtype=tf.float32
    )
    return tfio.experimental.filter.gaussian(x, d, k)

def randomRot90(x):
    k = tf.random.uniform(
        shape=[],
        minval=0,
        maxval=4,
        dtype=tf.dtypes.int32,
        seed=None,
        name=None
    )
    return tf.image.rot90(x, k)

def preprocess(x):
    y = tf.keras.layers.CenterCrop(224,224)(x)
    y = tf.keras.layers.Rescaling(1./127.5, offset=-1)(x)
    return y

def augment(x):
    y = randomGaussianBlur(x)
    y = randomRot90(x)
    return y

train_dataset = train_dataset.map(lambda x, y: (preprocess(x), y))
val_dataset = val_dataset.map(lambda x, y: (preprocess(x), y))
train_dataset = train_dataset.map(lambda x, y: (augment(x), y))

In [7]:
# Reference: https://www.tensorflow.org/tutorials/load_data/images#configure_the_dataset_for_performance
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
model = tf.keras.applications.Xception(
    include_top=False,
    weights=None,
    input_tensor=None,
    input_shape=(224,224,3),
    pooling=None, # hyperparameter?
)
#train_dataset = tf.keras.applications.xception.preprocess_input(train_dataset)
#val_dataset = tf.keras.applications.xception.preprocess_input(val_dataset)

In [9]:
opt = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=False,
)
loss = tf.keras.losses.BinaryCrossentropy(
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction=tf.keras.losses.Reduction.AUTO,
)
metric = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve='ROC',
    summation_method='interpolation',
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False
)
model.compile(
    optimizer=opt,
    loss=loss,
    metrics=metric
)

In [10]:
batch_size=32
epochs = 5
model.fit(
    train_dataset,
    batch_size=batch_size,
    epochs=epochs,
    verbose="auto",
    validation_data=val_dataset,
    shuffle=True,
    validation_freq=1
)

Epoch 1/5


ValueError: ignored